# Logistic Regression Consulting Project

## Binary Customer Churn

A marketing agency has many customers that use their service to produce ads for the client/customer websites. They've noticed that they have quite a bit of churn in clients. They basically randomly assign account managers right now, but want you to create a machine learning model that will help predict which customers will churn (stop buying their service) so that they can correctly assign the customers most at risk to churn an account manager. Luckily they have some historical data, can you help them out? Create a classification algorithm that will help classify whether or not a customer churned. Then the company can test this against incoming data for future customers to predict which customers will churn and assign them an account manager.

The data is saved as customer_churn.csv. Here are the fields and their definitions:

    Name : Name of the latest contact at Company
    Age: Customer Age
    Total_Purchase: Total Ads Purchased
    Account_Manager: Binary 0=No manager, 1= Account manager assigned
    Years: Totaly Years as a customer
    Num_sites: Number of websites that use the service.
    Onboard_date: Date that the name of the latest contact was onboarded
    Location: Client HQ Address
    Company: Name of Client Company
    
Once you've created the model and evaluated it, test out the model on some new data (you can think of this almost like a hold-out set) that your client has provided, saved under new_customers.csv. The client wants to know which customers are most likely to churn given this data (they don't have the label yet).

In [1]:
import findspark
findspark.init('/home/bahmansheikh/spark-3.1.2-bin-hadoop3.2')
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("bahman").getOrCreate()

22/01/01 11:18:25 WARN Utils: Your hostname, 1PNG6D3-XPS15 resolves to a loopback address: 127.0.1.1; using 192.168.1.140 instead (on interface wlp108s0)
22/01/01 11:18:25 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/01/01 11:18:25 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/01/01 11:18:26 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [4]:
data = spark.read.csv('customer_churn.csv',inferSchema=True,header=True)

In [10]:
testData = spark.read.csv('new_customers.csv',inferSchema=True,header=True)

In [5]:
data.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)



In [6]:
data.columns

['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn']

In [9]:
data.select(data['Churn']).show()

+-----+
|Churn|
+-----+
|    1|
|    1|
|    1|
|    1|
|    1|
|    1|
|    1|
|    1|
|    1|
|    1|
|    1|
|    1|
|    1|
|    1|
|    1|
|    1|
|    1|
|    1|
|    1|
|    1|
+-----+
only showing top 20 rows



In [11]:
testData.columns

['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company']

In [21]:
data.show()

+-------------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+
|              Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites|       Onboard_date|            Location|             Company|Churn|
+-------------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+
|   Cameron Williams|42.0|       11066.8|              0| 7.22|      8.0|2013-08-30 07:00:40|10265 Elizabeth M...|          Harvey LLC|    1|
|      Kevin Mueller|41.0|      11916.22|              0|  6.5|     11.0|2013-08-13 00:38:46|6157 Frank Garden...|          Wilson PLC|    1|
|        Eric Lozano|38.0|      12884.75|              0| 6.67|     12.0|2016-06-29 06:20:07|1331 Keith Court ...|Miller, Johnson a...|    1|
|      Phillip White|42.0|       8010.76|              0| 6.71|     10.0|2014-04-22 12:43:12|13120 Daniel Moun...|           Smith Inc|    1|
|     

In [16]:
data.groupBy('Location').count().show()

+--------------------+-----+
|            Location|count|
+--------------------+-----+
|062 Trevor Falls ...|    1|
|066 Jenkins Walks...|    1|
|45946 Day Springs...|    1|
|143 Andrea Flat L...|    1|
|Unit 2093 Box 153...|    1|
|399 Herbert Key P...|    1|
|104 Ruben Rapid A...|    1|
|930 Carrie Harbor...|    1|
|8202 Jade Unions ...|    1|
|USCGC Bailey FPO ...|    1|
|893 Carla Trace S...|    1|
|446 Rodney Ridge ...|    1|
|30668 Isabella Fr...|    1|
|911 Kent Point An...|    1|
|078 Nunez Haven S...|    1|
|PSC 5667, Box 831...|    1|
|4972 Michael Vill...|    1|
|567 Ian Loop Lamb...|    1|
|482 Wells Mountai...|    1|
|7259 Brown Street...|    1|
+--------------------+-----+
only showing top 20 rows



In [17]:
data.groupBy('Company').count().show()

+--------------------+-----+
|             Company|count|
+--------------------+-----+
|Miller, Johnson a...|    1|
|Hunter, Reyes and...|    1|
|          Obrien PLC|    1|
|            Soto PLC|    2|
|            Todd LLC|    1|
|Smith, Marshall a...|    1|
|           Smith PLC|    1|
|          Hall Group|    1|
|Freeman, Lam and ...|    1|
|       Smith-Carroll|    1|
|Hall, Hernandez a...|    1|
|          Cannon Inc|    1|
|        White-Dennis|    1|
|Wilson, Collins a...|    1|
|Jennings, Gates a...|    1|
|     Campbell-Willis|    1|
|    Martinez-Roberts|    1|
|        Robinson PLC|    1|
|          Barton Inc|    1|
|Hernandez, Middle...|    1|
+--------------------+-----+
only showing top 20 rows



In [35]:
from pyspark.ml.feature import (VectorAssembler,VectorIndexer,
                                OneHotEncoder,StringIndexer)
manager_indexer = StringIndexer(inputCol='Account_Manager',outputCol='managerIndex')
manager_encoder = OneHotEncoder(inputCol='managerIndex',outputCol='Manager')

In [25]:
from pyspark.sql.functions import year
data = data.withColumn("OnboardYear",year(data['Onboard_date']))

In [26]:
data.show()

+-------------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+-----------+
|              Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites|       Onboard_date|            Location|             Company|Churn|OnboardYear|
+-------------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+-----------+
|   Cameron Williams|42.0|       11066.8|              0| 7.22|      8.0|2013-08-30 07:00:40|10265 Elizabeth M...|          Harvey LLC|    1|       2013|
|      Kevin Mueller|41.0|      11916.22|              0|  6.5|     11.0|2013-08-13 00:38:46|6157 Frank Garden...|          Wilson PLC|    1|       2013|
|        Eric Lozano|38.0|      12884.75|              0| 6.67|     12.0|2016-06-29 06:20:07|1331 Keith Court ...|Miller, Johnson a...|    1|       2016|
|      Phillip White|42.0|       8010.76|              0| 6.71|     10.0|201

In [43]:
assembler = VectorAssembler(inputCols=['Age',
 'Total_Purchase',
 'Manager',
 'Years',
 'Num_Sites',
 'OnboardYear',],outputCol='features')

In [44]:
from pyspark.ml.classification import LogisticRegression

In [45]:
from pyspark.ml import Pipeline

In [46]:
log_reg_bahman = LogisticRegression(featuresCol='features',labelCol='Churn')

In [47]:
pipeline = Pipeline(stages=[manager_indexer, manager_encoder,assembler,log_reg_bahman])

In [41]:
data.show()

+-------------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+-----------+
|              Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites|       Onboard_date|            Location|             Company|Churn|OnboardYear|
+-------------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+-----------+
|   Cameron Williams|42.0|       11066.8|              0| 7.22|      8.0|2013-08-30 07:00:40|10265 Elizabeth M...|          Harvey LLC|    1|       2013|
|      Kevin Mueller|41.0|      11916.22|              0|  6.5|     11.0|2013-08-13 00:38:46|6157 Frank Garden...|          Wilson PLC|    1|       2013|
|        Eric Lozano|38.0|      12884.75|              0| 6.67|     12.0|2016-06-29 06:20:07|1331 Keith Court ...|Miller, Johnson a...|    1|       2016|
|      Phillip White|42.0|       8010.76|              0| 6.71|     10.0|201

In [48]:
fit_model = pipeline.fit(data)

22/01/01 12:10:02 ERROR LBFGS: Failure! Resetting history: breeze.optimize.FirstOrderException: Line search failed
22/01/01 12:10:02 ERROR LBFGS: Failure! Resetting history: breeze.optimize.FirstOrderException: Line search failed
22/01/01 12:10:03 ERROR LBFGS: Failure! Resetting history: breeze.optimize.FirstOrderException: Line search failed
22/01/01 12:10:03 ERROR LBFGS: Failure! Resetting history: breeze.optimize.FirstOrderException: Line search failed
22/01/01 12:10:03 ERROR LBFGS: Failure! Resetting history: breeze.optimize.FirstOrderException: Line search failed
22/01/01 12:10:03 ERROR LBFGS: Failure! Resetting history: breeze.optimize.FirstOrderException: Line search failed
22/01/01 12:10:03 ERROR LBFGS: Failure! Resetting history: breeze.optimize.FirstOrderException: Line search failed
22/01/01 12:10:03 ERROR LBFGS: Failure! Resetting history: breeze.optimize.FirstOrderException: Line search failed
22/01/01 12:10:04 ERROR LBFGS: Failure! Resetting history: breeze.optimize.First

22/01/01 12:10:14 ERROR LBFGS: Failure! Resetting history: breeze.optimize.FirstOrderException: Line search failed
22/01/01 12:10:14 ERROR LBFGS: Failure! Resetting history: breeze.optimize.FirstOrderException: Line search failed
22/01/01 12:10:14 ERROR LBFGS: Failure! Resetting history: breeze.optimize.FirstOrderException: Line search failed
22/01/01 12:10:14 ERROR LBFGS: Failure! Resetting history: breeze.optimize.FirstOrderException: Line search failed
22/01/01 12:10:15 ERROR LBFGS: Failure! Resetting history: breeze.optimize.FirstOrderException: Line search failed
22/01/01 12:10:15 ERROR LBFGS: Failure! Resetting history: breeze.optimize.FirstOrderException: Line search failed
22/01/01 12:10:15 ERROR LBFGS: Failure! Resetting history: breeze.optimize.FirstOrderException: Line search failed
22/01/01 12:10:15 ERROR LBFGS: Failure! Resetting history: breeze.optimize.FirstOrderException: Line search failed
22/01/01 12:10:15 ERROR LBFGS: Failure! Resetting history: breeze.optimize.First

In [52]:
testData = testData.withColumn("OnboardYear",year(testData['Onboard_date']))

In [53]:
results = fit_model.transform(testData)

In [54]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [55]:
my_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction',
                                       labelCol='Churn')

In [61]:
results.select('prediction').show()

+----------+
|prediction|
+----------+
|       0.0|
|       0.0|
|       0.0|
|       0.0|
|       0.0|
|       0.0|
+----------+

